# < 인스타그램 크롤링 - 메가커피 >

# 크롬 접속 및 인스타그램 접속

In [47]:
from selenium import webdriver #크롬접속용도
import time #쉬는시간설정용
import random #무작위로 쉬는시간 설정

# # 크롬창옵션설정 (셀레늄 크롬창 안보이게 설정)
# options = webdriver.ChromeOptions()
# options.add_argument('headless') 
# options.add_argument('window-size=1920x1080') 
# options.add_argument("disable-gpu")

# 크롬 접속하기 (안보이게 설정해뒀다)
driver = webdriver.Chrome('chromedriver.exe') #, #chrome_options=options   
driver.implicitly_wait(1000) #로딩중 1000초까지 기다리는데, 로딩이 완료되면 다음으로 넘어간다
                          
# 1. 인스타그램 접속하기
driver.get('https://www.instagram.com/')
driver.maximize_window()
time.sleep(random.uniform(1.1,2.5))

# 인스타그램 로그인

In [48]:
# 2.인스타계정으로 로그인
insta_id = 'redcross7777@naver.com'   
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0] #로그인창. input class 가져왔음
input_id.send_keys(insta_id)
time.sleep(1)

insta_pass = 'qwer1234@@qwer1234' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1] 
input_pw.send_keys(insta_pass)
time.sleep(1)

login = driver.find_element_by_css_selector('div.qF0y9.Igw0E.IwRSH.eGOV_._4EzTm.bkEs3.CovQj.jKUp7.DhRcB > button') 
login.click() # div밑의 button class 가져왔음
time.sleep(random.uniform(2,4))

# 인스타그램 주소 형성

In [49]:
# 3.인스타주소만들기 
def insta_search(keyword): 
    insta_url = 'https://www.instagram.com/explore/tags/'+ keyword
    return insta_url

# 주소에 메가커피 키워드 넣고 실행

In [50]:
# 3.실행
keyword='메가커피'
url = insta_search(keyword)
driver.implicitly_wait(1000) 
driver.get(url)
time.sleep(random.uniform(2,4))

# 첫번째 사진 클릭 실행

In [51]:
# 4. 첫번째 사진클릭 함수
def click_first(driver): 
    driver.implicitly_wait(1000)
    first_pic = driver.find_element_by_css_selector('div._9AhH0') # div주소
    first_pic.click() 
    time.sleep(random.uniform(1.1,2.5))

In [52]:
# 4.실행
click_first(driver)

# 본문 가져오기

In [53]:
# 5. 본문 가져오는 함수

import re #정규표현식 분석용
from bs4 import BeautifulSoup # html분석용

# from selenium.webdriver.support.ui import WebDriverWait #조건쉬는시간설정용
# from selenium.webdriver.common.by import By #iw 줄여쓰기용
# from selenium.webdriver.support import expected_conditions as EC

# wait= WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.C4VMK")))
# #본문내용 보일때까지 대기

def insta_content(driver):
    # 1) 본문
    driver.implicitly_wait(1000)
    html = driver.page_source
    time.sleep(random.uniform(1.1,2.5)) 
    soup = BeautifulSoup(html, 'lxml') # xml파일 해석하는 파이썬패키지 lxml   
    main_text = soup.select('div.C4VMK > span')[0].text  #div밑의 span목록 가져왔음 

    #[<span class="">やっと休日がきたぞ。😆<br/><br/>드디어 휴일이 찾아왔다구 -  
    #[0]인덱스지정안하면 리스트로 반환되버림
    #본문은 항상있다


    # 2) 해쉬태그
    tags= re.findall(r'#[^\s#]+', main_text)  #본문에서 앞에 공백있고 #인 단어가 1번이상 나온 문자 전부출력
    #해쉬태그는 항상있다 ['#テクテクのんびり絵日記', '#테구테구그림일기']

    # 3) 작성일자
    write_date = soup.select('div.k_Q0X.I0_K8.NnvRN > a > time')[0]['datetime'][:10] #div밑의 time class 가져옴
    #한줄짜리. <time class="_1o9PC Nzb55" datetime="2021-07-21T09:40:50.000Z" title="2021년 7월 21일">7월 21일</time>
    #datetime="2021-07-21T09:40:50.000Z"
    #작성일자는 항상있다 

    # 4) 좋아요 수

    try:
        like = soup.select('div.Nm9Fw  > a')[0].text[4:-1]  #div밑의 a class 가져옴
    except:
        like = 0 # 좋아요가 없을때  
        
    #한줄짜리.[<a class="zV_Nj" href="/p/CRlaSyXBxQh/liked_by/" tabindex="0">좋아요 <span>262</span>개</a>]
    #'좋아요 262개'
    

    # 5) 위치정보

    try: 
        place = soup.select('div.JF9hh > a')[0].text  #div밑의 a class 가져옴
    except:
        place = '' #위치정보 없을때

    #한줄짜리.[<a class="O4GlU" href="/explore/locations/270184519803398/" tabindex="0">녹양동</a>]

    # 6) 전체내용 저장 
    output = [main_text, tags, write_date , like, place, ]

    return output

# 가져온 본문 데이터 확인

In [54]:
# 5.본문데이터 테스트
insta_content(driver)

['📍#메가커피 요즘 어딜 가나 보이는 메가커피..!정류장마다 한 곳씩은 있는거 같아요😆-🧾퐁크러쉬 플레인- 3.9메가커피에서 젤 사랑하는 메뉴❣️칼로리 생각 안하고 요즘은 일주일에 한 번은 마시는중🤦🏻\u200d♀️ 맛있으니 우째,, 마셔야지 모🙄맛은 조리퐁을 우유에 말아먹는 것보다 세배는 맛있는 달달한 그 맛!위에 조리퐁 얹어줘서 눅눅해지기 전에 빨대로 흡입해야함😙맛은 플레인, 바나나, 딸기 이렇게 있지만 아무리 퐁크러쉬를 사랑한다해도 플레인 외에는 시도하기가 참😅너무 신상 리뷰처럼 써놓았네요.. 다들 드셔보셨죠..?😏 메가커피의 장점은 뭐니뭐니해도 가성비인거 같아요! 일반 카페들보다 컵이 정말 크죠ㅎㅎ 쨌든 앞으로도 전 퐁크러쉬만 먹으러 가겠습니다🤙🏼#퐁크러쉬 #바나나퐁크러쉬 #딸기퐁크러쉬 #조리퐁 #조리퐁라떼 #메가커피추천 #메가커피메뉴 #카페 #카페추천 #카페스타그램 #카페투어 #카페맛집 #카페디저트 #먹스타그램 #먹스타맞팔 #먹스타그램맞팔 #먹스타소통 #맛스타그램 #푸드스타그램 #맞팔 #맛팔 #먹팔 #먹팔해요 #먹팔환영 #맛집 #맛집추천 #맛집스타그램 #맛집탐방 #맛집투어',
 ['#메가커피',
  '#퐁크러쉬',
  '#바나나퐁크러쉬',
  '#딸기퐁크러쉬',
  '#조리퐁',
  '#조리퐁라떼',
  '#메가커피추천',
  '#메가커피메뉴',
  '#카페',
  '#카페추천',
  '#카페스타그램',
  '#카페투어',
  '#카페맛집',
  '#카페디저트',
  '#먹스타그램',
  '#먹스타맞팔',
  '#먹스타그램맞팔',
  '#먹스타소통',
  '#맛스타그램',
  '#푸드스타그램',
  '#맞팔',
  '#맛팔',
  '#먹팔',
  '#먹팔해요',
  '#먹팔환영',
  '#맛집',
  '#맛집추천',
  '#맛집스타그램',
  '#맛집탐방',
  '#맛집투어'],
 '

# 다음 페이지 넘어가기(우클릭) 실행

In [55]:
# 6. 다음페이지 넘어가는 함수 (오른쪽클릭)

from selenium.webdriver.common.keys import Keys #send_keys 내장패키지

def click_next(driver):
    driver.implicitly_wait(1000)    
    next_page = driver.find_element_by_css_selector('div.DdSX2 > a._65Bje.coreSpriteRightPaginationArrow') #div밑의 a class 가져옴    
    next_page.send_keys(Keys.ARROW_RIGHT)  # next_page.click() 에서 교체,
    time.sleep(random.uniform(1.1,2.5))  

'''
증상: next_page.click()로 1300개째 긁어오는중 해당 오류발생.

Message: element click intercepted: Element <a class=" _65Bje    coreSpriteRightPaginationArrow" tabindex="0">...
</a> is not clickable at point (1096, 302). Other element would receive the click: <span class="">...</span>

해결: 구글링해서 click대신 send_keys 로 바꿔줌.

@DonSimon I had the same issue that you did. I was able to get around it by using .
SendKeys(Keys.Return) instead of .Click. This worked for me in Chrome where I was having this issue and in Firefox where I was having another similar issue on this particular link (anchor wrapped in a Div). – 

https://stackoverflow.com/questions/38521136/python-selenium-attributeerror-webelement-object-has-no-attribute-sendkeys
'''


'\n증상: next_page.click()로 1300개째 긁어오는중 해당 오류가 났다.\n\nMessage: element click intercepted: Element <a class=" _65Bje    coreSpriteRightPaginationArrow" tabindex="0">...\n</a> is not clickable at point (1096, 302). Other element would receive the click: <span class="">...</span>\n\n해결: 구글링해서 click대신 send_keys 로 바꿔주었다\n\n@DonSimon I had the same issue that you did. I was able to get around it by using .\nSendKeys(Keys.Return) instead of .Click. This worked for me in Chrome where I was having this issue and in Firefox where I was having another similar issue on this particular link (anchor wrapped in a Div). – \n\nhttps://stackoverflow.com/questions/38521136/python-selenium-attributeerror-webelement-object-has-no-attribute-sendkeys\n'

In [56]:
# 6. 실행
click_next(driver)

# 여러 페이지 크롤링

In [62]:
# 7. 여러페이지 가져오기 (크롤링)
import pandas as pd

number = 110000    #크롤링 갯수 지정
insta_result = [ ]   #데이터 저장할 리스트
insta_count=0        #크롤링 카운트


for i in range(number):
    
    # 게시글 수집에서 오류나면 쉬고 다음페이지에서 계속하도록 함
    try:
        insta_count +=1
        insta_output = insta_content(driver)   #내용가져오고
        insta_result.append(insta_output)      #리스트에저장후
        print(insta_count,'번째크롤링')
        
        if insta_count %1000 == 0 :  #1000개마다 현재까지 크롤링 데이터 csv저장
            df= pd.DataFrame(insta_result) 
            df.columns = ['본문','해쉬태그','작성시간','좋아요','위치'] #컬럼명지정
            df.to_csv('./csv/insta_megacoffee_{}.csv'.format(insta_count), encoding = 'utf-8-sig', index=False)      
            insta_result=[ ] #2000개이상 크롤링시 데이터를 더 가져오지 못해서, csv저장후 리스트초기화 시켰다. 
            print(insta_result)        
        
        time.sleep( random.uniform(5.7,10.3) ) #쉬고난후
        click_next(driver)             #다음페이지로   
        
    except:
        insta_count +=1
        time.sleep( random.uniform(5.7,10.3) )
        click_next(driver) 
        
'''    
증상 : 4500개째에서 오류발생(웹로딩시간지연)
해결 :  driver.implicitly_wait(1000) 추가
#로딩이 완전히 끝나고 연산하도록 설정

4589 번째크롤링
---------------------------------------------------------------------------
<ipython-input-7-90026ab1e6b6> in insta_content(driver)
     11 
---> 12     main_text = soup.select('div.C4VMK > span')[0].text  #div밑의 span목록 가져옴.
     13 
     
IndexError: list index out of range

During handling of the above exception, another exception occurred:
'''

1 번째크롤링
2 번째크롤링
3 번째크롤링
4 번째크롤링
5 번째크롤링
6 번째크롤링
7 번째크롤링
8 번째크롤링
9 번째크롤링
10 번째크롤링
11 번째크롤링
12 번째크롤링
13 번째크롤링
14 번째크롤링
15 번째크롤링
16 번째크롤링
17 번째크롤링
18 번째크롤링
19 번째크롤링
20 번째크롤링
21 번째크롤링
22 번째크롤링
23 번째크롤링
24 번째크롤링
25 번째크롤링
26 번째크롤링
27 번째크롤링
28 번째크롤링
29 번째크롤링
30 번째크롤링
31 번째크롤링
32 번째크롤링
33 번째크롤링
34 번째크롤링
35 번째크롤링
36 번째크롤링
37 번째크롤링
38 번째크롤링
39 번째크롤링
40 번째크롤링
41 번째크롤링
42 번째크롤링
43 번째크롤링
44 번째크롤링
45 번째크롤링
46 번째크롤링
47 번째크롤링
48 번째크롤링
49 번째크롤링
50 번째크롤링
51 번째크롤링
52 번째크롤링
53 번째크롤링
54 번째크롤링
55 번째크롤링
56 번째크롤링
57 번째크롤링
58 번째크롤링
59 번째크롤링
60 번째크롤링
61 번째크롤링
62 번째크롤링
63 번째크롤링
64 번째크롤링
65 번째크롤링
66 번째크롤링
67 번째크롤링
68 번째크롤링
69 번째크롤링
70 번째크롤링
71 번째크롤링
72 번째크롤링
73 번째크롤링
74 번째크롤링
75 번째크롤링
76 번째크롤링
77 번째크롤링
78 번째크롤링
79 번째크롤링
80 번째크롤링
81 번째크롤링
82 번째크롤링
83 번째크롤링
84 번째크롤링
85 번째크롤링
86 번째크롤링
87 번째크롤링
88 번째크롤링
89 번째크롤링
90 번째크롤링
91 번째크롤링
92 번째크롤링
93 번째크롤링
94 번째크롤링
95 번째크롤링
96 번째크롤링
97 번째크롤링
98 번째크롤링
99 번째크롤링
100 번째크롤링
101 번째크롤링
102 번째크롤링
103 번째크롤링
104 번째크롤링
105 번째크롤링
106 번째크롤링
107 번째크롤링
108 번째크롤링
109 번째크롤링
110 번째크롤링
111 번째크롤

908 번째크롤링
909 번째크롤링
910 번째크롤링
911 번째크롤링
912 번째크롤링
913 번째크롤링
914 번째크롤링
915 번째크롤링
916 번째크롤링
917 번째크롤링
918 번째크롤링
919 번째크롤링
920 번째크롤링
921 번째크롤링
922 번째크롤링
923 번째크롤링
924 번째크롤링
925 번째크롤링
926 번째크롤링
927 번째크롤링
928 번째크롤링
929 번째크롤링
930 번째크롤링
931 번째크롤링
932 번째크롤링
933 번째크롤링
934 번째크롤링
935 번째크롤링
936 번째크롤링
937 번째크롤링
938 번째크롤링
941 번째크롤링
942 번째크롤링
943 번째크롤링
944 번째크롤링
945 번째크롤링
946 번째크롤링
947 번째크롤링
948 번째크롤링
949 번째크롤링
950 번째크롤링
951 번째크롤링
952 번째크롤링
953 번째크롤링
954 번째크롤링
955 번째크롤링
956 번째크롤링
957 번째크롤링
958 번째크롤링
959 번째크롤링
960 번째크롤링
961 번째크롤링
962 번째크롤링
963 번째크롤링
964 번째크롤링
965 번째크롤링
966 번째크롤링
967 번째크롤링
968 번째크롤링
969 번째크롤링
970 번째크롤링
971 번째크롤링
972 번째크롤링
973 번째크롤링
974 번째크롤링
975 번째크롤링
976 번째크롤링
977 번째크롤링
978 번째크롤링
979 번째크롤링
980 번째크롤링
981 번째크롤링
982 번째크롤링
983 번째크롤링
984 번째크롤링
985 번째크롤링
986 번째크롤링
987 번째크롤링
988 번째크롤링
989 번째크롤링
990 번째크롤링
991 번째크롤링
992 번째크롤링
993 번째크롤링
994 번째크롤링
995 번째크롤링
996 번째크롤링
997 번째크롤링
998 번째크롤링
999 번째크롤링
1000 번째크롤링
[]
1003 번째크롤링
1004 번째크롤링
1005 번째크롤링
1006 번째크롤링
1007 번째크롤링
1008 번째크롤링
1009 번째크롤링
1010 번째크롤

1675 번째크롤링
1676 번째크롤링
1677 번째크롤링
1678 번째크롤링
1679 번째크롤링
1680 번째크롤링
1681 번째크롤링
1682 번째크롤링
1683 번째크롤링
1684 번째크롤링
1685 번째크롤링
1686 번째크롤링
1687 번째크롤링
1688 번째크롤링
1689 번째크롤링
1690 번째크롤링
1691 번째크롤링
1692 번째크롤링
1693 번째크롤링
1694 번째크롤링
1695 번째크롤링
1696 번째크롤링
1697 번째크롤링
1698 번째크롤링
1699 번째크롤링
1700 번째크롤링
1701 번째크롤링
1702 번째크롤링
1703 번째크롤링
1704 번째크롤링
1705 번째크롤링
1706 번째크롤링
1707 번째크롤링
1708 번째크롤링
1709 번째크롤링
1710 번째크롤링
1711 번째크롤링
1712 번째크롤링
1713 번째크롤링
1714 번째크롤링
1715 번째크롤링
1716 번째크롤링
1717 번째크롤링
1718 번째크롤링
1719 번째크롤링
1720 번째크롤링
1721 번째크롤링
1722 번째크롤링
1723 번째크롤링
1724 번째크롤링
1725 번째크롤링
1726 번째크롤링
1727 번째크롤링
1728 번째크롤링
1729 번째크롤링
1730 번째크롤링
1731 번째크롤링
1732 번째크롤링
1733 번째크롤링
1734 번째크롤링
1735 번째크롤링
1736 번째크롤링
1737 번째크롤링
1738 번째크롤링
1739 번째크롤링
1740 번째크롤링
1741 번째크롤링
1742 번째크롤링
1743 번째크롤링
1744 번째크롤링
1745 번째크롤링
1746 번째크롤링
1747 번째크롤링
1748 번째크롤링
1749 번째크롤링
1750 번째크롤링
1751 번째크롤링
1752 번째크롤링
1753 번째크롤링
1754 번째크롤링
1755 번째크롤링
1756 번째크롤링
1757 번째크롤링
1758 번째크롤링
1759 번째크롤링
1760 번째크롤링
1761 번째크롤링
1762 번째크롤링
1763 번째크롤링
1764 번째크롤링
1765 번째크롤링

2430 번째크롤링
2431 번째크롤링
2432 번째크롤링
2433 번째크롤링
2434 번째크롤링
2435 번째크롤링
2436 번째크롤링
2437 번째크롤링
2438 번째크롤링
2439 번째크롤링
2440 번째크롤링
2441 번째크롤링
2442 번째크롤링
2443 번째크롤링
2444 번째크롤링
2445 번째크롤링
2446 번째크롤링
2447 번째크롤링
2448 번째크롤링
2449 번째크롤링
2450 번째크롤링
2451 번째크롤링
2452 번째크롤링
2453 번째크롤링
2454 번째크롤링
2455 번째크롤링
2456 번째크롤링
2457 번째크롤링
2458 번째크롤링
2459 번째크롤링
2460 번째크롤링
2461 번째크롤링
2462 번째크롤링
2463 번째크롤링
2464 번째크롤링
2465 번째크롤링
2466 번째크롤링
2467 번째크롤링
2468 번째크롤링
2469 번째크롤링
2470 번째크롤링
2471 번째크롤링
2472 번째크롤링
2473 번째크롤링
2474 번째크롤링
2475 번째크롤링
2476 번째크롤링
2477 번째크롤링
2478 번째크롤링
2479 번째크롤링
2480 번째크롤링
2481 번째크롤링
2482 번째크롤링
2483 번째크롤링
2484 번째크롤링
2485 번째크롤링
2486 번째크롤링
2487 번째크롤링
2488 번째크롤링
2489 번째크롤링
2490 번째크롤링
2491 번째크롤링
2492 번째크롤링
2493 번째크롤링
2494 번째크롤링
2495 번째크롤링
2496 번째크롤링
2497 번째크롤링
2498 번째크롤링
2499 번째크롤링
2500 번째크롤링
2501 번째크롤링
2502 번째크롤링
2503 번째크롤링
2504 번째크롤링
2505 번째크롤링
2506 번째크롤링
2507 번째크롤링
2508 번째크롤링
2509 번째크롤링
2510 번째크롤링
2511 번째크롤링
2512 번째크롤링
2513 번째크롤링
2514 번째크롤링
2515 번째크롤링
2516 번째크롤링
2517 번째크롤링
2518 번째크롤링
2519 번째크롤링
2520 번째크롤링

3179 번째크롤링
3180 번째크롤링
3181 번째크롤링
3182 번째크롤링
3183 번째크롤링
3184 번째크롤링
3185 번째크롤링
3186 번째크롤링
3187 번째크롤링
3188 번째크롤링
3189 번째크롤링
3190 번째크롤링
3191 번째크롤링
3192 번째크롤링
3193 번째크롤링
3194 번째크롤링
3195 번째크롤링
3196 번째크롤링
3197 번째크롤링
3198 번째크롤링
3199 번째크롤링
3200 번째크롤링
3201 번째크롤링
3202 번째크롤링
3203 번째크롤링
3204 번째크롤링
3205 번째크롤링
3206 번째크롤링
3207 번째크롤링
3208 번째크롤링
3209 번째크롤링
3210 번째크롤링
3211 번째크롤링
3212 번째크롤링
3213 번째크롤링
3214 번째크롤링
3215 번째크롤링
3216 번째크롤링
3217 번째크롤링
3218 번째크롤링
3219 번째크롤링
3220 번째크롤링
3221 번째크롤링
3222 번째크롤링
3223 번째크롤링
3224 번째크롤링
3225 번째크롤링
3226 번째크롤링
3227 번째크롤링
3228 번째크롤링
3229 번째크롤링
3230 번째크롤링
3231 번째크롤링
3232 번째크롤링
3233 번째크롤링
3234 번째크롤링
3235 번째크롤링
3236 번째크롤링
3237 번째크롤링
3238 번째크롤링
3239 번째크롤링
3240 번째크롤링
3241 번째크롤링
3242 번째크롤링
3243 번째크롤링
3244 번째크롤링
3245 번째크롤링
3246 번째크롤링
3247 번째크롤링
3248 번째크롤링
3249 번째크롤링
3250 번째크롤링
3251 번째크롤링
3252 번째크롤링
3253 번째크롤링
3254 번째크롤링
3255 번째크롤링
3256 번째크롤링
3257 번째크롤링
3258 번째크롤링
3259 번째크롤링
3260 번째크롤링
3261 번째크롤링
3262 번째크롤링
3263 번째크롤링
3264 번째크롤링
3265 번째크롤링
3266 번째크롤링
3267 번째크롤링
3268 번째크롤링
3269 번째크롤링

AttributeError: 'NoneType' object has no attribute 'send_keys'

# < 전처리 과정 >

# 결과값 데이터프레임 형성

In [64]:
#8. 결과값 df로 만들기
import pandas as pd

df= pd.DataFrame(insta_result)
df.columns = ['본문','해쉬태그','작성시간','좋아요','위치'] #컬럼명지정

In [65]:
df.to_csv('insta_megacoffee_320.csv', encoding = 'utf-8-sig', index=False) #csv로 저장

#index=True 이면 앞에 열이 하나 더 추가되어 저장됨

# 1000개 단위로 크롤링해서 csv로 저장

In [ ]:
# 크롤링시 인스타그램 서버에서 1000개 이상 게시물 크롤링시 오류발생
# 문제해결을 위해 1000개 단위로 크롤링해서 저장함
# 저장된 개별파일을 하나로 합치는 과정을 아래코드에 구현함

In [77]:
df1=pd.read_csv('csv/insta_megacoffee_1000.csv')
df1.head(5) #나눠서 저장한 csv파일 불러옴

본문  \
0  📍#메가커피 요즘 어딜 가나 보이는 메가커피..!정류장마다 한 곳씩은 있는거 같아요...   
1  ㅡ🦋#간만에 업뎃,#레몬자몽라임#메가에이드 🍹 마시며#상쾌한하루를시작 ‼️#날씨도좋...   
2  디저트는 빠질수 없지 😎 ..#오늘#저녁#허자블#매직에이드#아아#감자빵#폭주#다욧오...   
3  구사언니.📍#메가커피✔️죠리퐁 #플레인퐁크러쉬 3.9.나에겐 너무 큰… 죠리퐁..플...   
4  우리동네도 생긴 메가커피💕퐁크러쉬 너무조아!#우리동네도 #생긴 #메가커피 #최애메뉴...   

                                                해쉬태그        작성시간  좋아요  \
0  ['#메가커피', '#퐁크러쉬', '#바나나퐁크러쉬', '#딸기퐁크러쉬', '#조리...  2021-08-09  427   
1  ['#간만에', '#레몬자몽라임', '#메가에이드', '#상쾌한하루를시작', '#날...  2021-08-05  308   
2  ['#오늘', '#저녁', '#허자블', '#매직에이드', '#아아', '#감자빵'...  2021-09-30  167   
3  ['#메가커피✔️죠리퐁', '#플레인퐁크러쉬', '#맛집', '#신사카페', '#강...  2021-06-24  414   
4  ['#우리동네도', '#생긴', '#메가커피', '#최애메뉴', '#풍크러쉬', '...  2021-09-30  184   

                                   위치  
0                                메가커피  
1                                 NaN  
2                                 NaN  
3  Garosu-gil / Sinsa-dong Garosu-gil  
4                                 NaN

In [76]:
df2=pd.read_csv('csv/insta_megacoffee_2000.csv')
df2.head(5)

본문  \
0  #10월둘째주기록 #마라탕 #꿔바로우 #메가커피 #딸기라떼 #북악스카웨이한강라면 #...   
1  🛍롯데하이마트 석촌역점안마의자 주말 원데이 릴레이 초특가하루에 한모델 파격세일! 0...   
2  홍성명동상가SNS서포터즈의생생 후기가 도착했어요!💌자세한 사항은 홍성명동상가공식블로...   
3  가족들과 가을 나들이로 합천에 루지🏎 타고 왔어요😆길이나 경사도는 어느정도 스릴도 ...   
4                                              #메가커피   

                                                해쉬태그        작성시간  좋아요  \
0  ['#10월둘째주기록', '#마라탕', '#꿔바로우', '#메가커피', '#딸기라떼...  2021-10-11    5   
1  ['#주목!', '#롯데', '#석촌역점', '#휴대폰', '#갤럭시Z', '#Z폴...  2021-10-11    2   
2  ['#홍성', '#홍성명동상가', '#SNS서포터즈', '#서포터즈', '#생생후기...  2021-10-11   21   
3  ['#합천', '#합천영상테마파크▪합천', '#신소양체육공원', '#합천여행', '...  2021-10-10  134   
4                                          ['#메가커피']  2021-10-10    1   

            위치  
0          NaN  
1          NaN  
2       홍성명동상가  
3  합천국보테마파크 루지  
4          NaN

In [80]:
df3=pd.read_csv('csv/insta_megacoffee_3000.csv')
df3.head(5)

본문  \
0                           가을전어맛있어#오징어게임#달고나뽑기#메가커피   
1                           _ #아아로해장🥤 #당분간막걸리랑이별예정이요   
2  #메가커피#가성비최고 #뿌염후#동네친구랑👭 #커피한잔☕️ #싸워도#36년째친구🤗 #...   
3  오늘 엄마랑 아빠랑 나랑 같이 점심밥 나는 메가에이드 샀다 그리고 마당촌 고기집으로...   
4  동네에 메가커피 생겨서한번 먹어봄 😄가성비는 좋으네~~ 👏🏻🥤#아아 #아메리카노 #...   

                                                해쉬태그        작성시간  좋아요    위치  
0                      ['#오징어게임', '#달고나뽑기', '#메가커피']  2021-09-26    9   NaN  
1                      ['#아아로해장🥤', '#당분간막걸리랑이별예정이요']  2021-09-26   58   NaN  
2  ['#메가커피', '#가성비최고', '#뿌염후', '#동네친구랑👭', '#커피한잔☕...  2021-09-26  290   NaN  
3                       ['#메가커피', '#한우맛집', '#메가에이드']  2021-09-26   11  메가커피  
4  ['#아아', '#아메리카노', '#커피', '#드라이브', '#힐링', '#커피한...  2021-09-26  140   NaN

In [81]:
df4=pd.read_csv('csv/insta_megacoffee_320.csv')
df4.head(5)

본문  \
0  가을 시즌 메뉴 두둥등장!!!정_말 하나같이 다 주옥같은 맛들..!여러분의 입맛을 ...   
1  바다로 놀러가진 못해도 메가커피에서 바다 본다!!⭐️블루하와이⭐️우유와 커피, 곡물...   
2  주말 아침은 카페에서 𖤐일요일 주말 아침이라서 집앞 빵집에 가서 간단하게 빵으로 아...   
3  만나면 밤새는줄 모르는 갓파더 브라더스 👍#갓파더 #godfather #부산할리 #...   
4  #디카페인커피 #쿠폰 #비오는날 #메가커피 #완주첨단 #봉동 #테이크아웃 #배달카페...   

                                                해쉬태그        작성시간  좋아요  \
0  ['#메가커피신메뉴', '#메가커피가을신메뉴', '#메가커피', '#원주기업도시',...  2021-09-12   13   
1  ['#블루하와이', '#메가커피아이스크림', '#메가커피', '#원주기업도시', '...  2021-09-12   22   
2  ['#일요일', '#아침', '#메가커피', '#MEGAMGCCOFFEE', '#아...  2021-09-12   21   
3  ['#갓파더', '#godfather', '#부산할리', '#할리부산', '#부산바...  2021-09-12  181   
4  ['#디카페인커피', '#쿠폰', '#비오는날', '#메가커피', '#완주첨단', ...  2021-09-12    7   

                 위치  
0            원주기업도시  
1            원주기업도시  
2               NaN  
3               NaN  
4  전라북도 완주군 봉동읍 둔산리

# 1000개 단위 저장 문서 하나로 합치기

In [83]:
#개별문서 concat로 합치기
df5 = pd.concat([df1,df2,df3,df4], ignore_index=True)
df5

df5.to_csv('./csv/insta_megacoffee_complete.csv'.format(insta_count), encoding = 'utf-8-sig', index=False)
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   본문      3218 non-null   object
 1   해쉬태그    3218 non-null   object
 2   작성시간    3218 non-null   object
 3   좋아요     3218 non-null   object
 4   위치      718 non-null    object
dtypes: object(5)
memory usage: 125.8+ KB


In [84]:
df5

본문  \
0     📍#메가커피 요즘 어딜 가나 보이는 메가커피..!정류장마다 한 곳씩은 있는거 같아요...   
1     ㅡ🦋#간만에 업뎃,#레몬자몽라임#메가에이드 🍹 마시며#상쾌한하루를시작 ‼️#날씨도좋...   
2     디저트는 빠질수 없지 😎 ..#오늘#저녁#허자블#매직에이드#아아#감자빵#폭주#다욧오...   
3     구사언니.📍#메가커피✔️죠리퐁 #플레인퐁크러쉬 3.9.나에겐 너무 큰… 죠리퐁..플...   
4     우리동네도 생긴 메가커피💕퐁크러쉬 너무조아!#우리동네도 #생긴 #메가커피 #최애메뉴...   
...                                                 ...   
3213  🍁🍁🍁🍁🍁🍁메가커피 가을신메뉴~🍂그냥 하는말이 아니고 진짜 맛있는 맛🧋🧋🧋아는맛이 ...   
3214  #내돈내산 #메가커피 #유니콘마카롱메가커피 마카롱인데 너무 맛있어요 😭😭 크기도 크...   
3215  ㅡ쥬스먹으러간다는데할말없이 기여움ㅋㅋ뚜주니는  딸기쥬스유주니는 수박쥬스 주세요ㅋㅋ#...   
3216  -#자부타임 #메가커피 #카라멜마끼아또 #나에게쓰는편지 #얼갈이겉절이 #고메탕수육 ...   
3217                                   #메가커피 #사과유자차 #05   

                                                   해쉬태그        작성시간  좋아요  \
0     ['#메가커피', '#퐁크러쉬', '#바나나퐁크러쉬', '#딸기퐁크러쉬', '#조리...  2021-08-09  427   
1     ['#간만에', '#레몬자몽라임', '#메가에이드', '#상쾌한하루를시작', '#날...  2021-08-05  308   
2     ['#오늘', '#저녁', '#허자블', '#매직에이드', '#아아', '#감자빵'...  2021-09-30  167   
3     ['#메가커피✔️죠리퐁', '#플레인퐁크러쉬', '#맛집', '#신사카페', '#강...  2021-06-24  414   
4     ['#우리동네도', '#생긴', '#메가커피', '#최애메뉴', '#풍크러쉬', '...  2021-09-30  184   
...                                                 ...         ...  ...   
3213  ['#메가커피', '#세종시메가커피', '#세종시맛집', '#여름시즌메뉴', '#코...  2021-09-06   24   
3214                  ['#내돈내산', '#메가커피', '#유니콘마카롱메가커피']  2021-09-06  272   
3215                      ['#메가커피', '#오룡메가커피', '#기요미둘']  2021-09-06   25   
3216  ['#자부타임', '#메가커피', '#카라멜마끼아또', '#나에게쓰는편지', '#얼...  2021-09-04   20   
3217                         ['#메가커피', '#사과유자차', '#05']  2021-09-04   53   

                                      위치  
0                                   메가커피  
1                                    NaN  
2                                    NaN  
3     Garosu-gil / Sinsa-dong Garosu-gil  
4                                    NaN  
...                                  ...  
3213                                 NaN  
3214                        Seoul, Korea  
3215                                 NaN  
3216                                 NaN  
3217                                 NaN  

[3218 rows x 5 columns]

# < 워드클라우드로 해쉬태그 시각화 >

# 해쉬태그 column만 가져와서 하나의 데이터프레임으로 합치기

In [85]:
# 1.해쉬태그시각화하기 (워드클라우드사용)

df5['해쉬태그']

0       ['#메가커피', '#퐁크러쉬', '#바나나퐁크러쉬', '#딸기퐁크러쉬', '#조리...
1       ['#간만에', '#레몬자몽라임', '#메가에이드', '#상쾌한하루를시작', '#날...
2       ['#오늘', '#저녁', '#허자블', '#매직에이드', '#아아', '#감자빵'...
3       ['#메가커피✔️죠리퐁', '#플레인퐁크러쉬', '#맛집', '#신사카페', '#강...
4       ['#우리동네도', '#생긴', '#메가커피', '#최애메뉴', '#풍크러쉬', '...
                              ...                        
3213    ['#메가커피', '#세종시메가커피', '#세종시맛집', '#여름시즌메뉴', '#코...
3214                    ['#내돈내산', '#메가커피', '#유니콘마카롱메가커피']
3215                        ['#메가커피', '#오룡메가커피', '#기요미둘']
3216    ['#자부타임', '#메가커피', '#카라멜마끼아또', '#나에게쓰는편지', '#얼...
3217                           ['#메가커피', '#사과유자차', '#05']
Name: 해쉬태그, Length: 3218, dtype: object

In [125]:
# 2.df 해쉬태그 하나로 합치기

hash_tags = []
for i in df['해쉬태그']:
    for j in i:
        hash_tags.append(j)

print(len(hash_tags))
print(type(hash_tags))

3583
<class 'list'>


# 불용어 지정 (수작업)

In [194]:
# 3.불용어 지정 (워드클라우드 작성)
stop_words = ['#메가커피' , '#megacoffee', '#MEGACOFFEE', '#cafe', '#coffee', '#먹스타그램', '#카페', \
             '#카페스타그램', '#커피스타그램', '#먹팔', '#일상', '#커피', '#월요일','#화요일','#수요일', \
             '#목요일','#금요일','#토요일','#일요일', '#먹스타', '#메가', '#맞팔', '#셀카', '#소통', '#맞팔',\
             '#일상스타그램','#선팔하면맞팔', '#좋아요반사', '#먹스타맞팔', '#선팔', '#맛스타그램', '#집사맞팔'\
             '#커피그램','#좋반','#팔로우환영','#일상그램','#ㅈㅗㅎㅇㅏㅇㅛㅂㅏㄴㅅㅏ','#메가커피추천','#소통해요',\
             '#일상소통','#먹방','#맞팔선팔환영','#데일리그램','#맞팔해요','#좋아요테러','#일상기록', \
             '#인친','#ㅈㅗㅎㅇ ㅏㅇㅛㅂ ㅏㄴㅅ ㅏ','#회원모집','#선팔맞팔','#팔로우','#럽스타그램','#메가커피애용자',\
             '#좋아요','#사랑해','#벌써겨울','#주말일상','#주말','#여행그램','#2021', '#9', '#1', '#💑', \
             '#금남시장','#daily','#😜','#금호','#모소리','#데이트','#🥪','#l4l','#👫','#💕','#사진','#데일리', \
             '#coffeestagram','#f4f','#맞팔환영','#likeforlikes','#fff','#followforfollowback','#신경정신과', \
             '#모소리살', '#가오리살', '#껍데기', '#짜글이밥','#팔로워', '#러브스타그램', '#러브스타그램', '#coffeetime', \
             '#충무로','#메가커피충북대점', '#다산기공', '#원주기업도시', '#돼지특수부위전문점', '#수유', '#화계', \
             '#어진이골목시장','#성신여대', '#레드라인', '#완주첨단', '#봉동', '#육군훈련소카페', '#입대', \
             '#연무고등학교','#육군훈련소', '#f4l', '#cafestagram', '#cafetour', '#cafelover', '#메가커피🥰',\
             '#감자탕맛집', '#선팔환영','#내돈내산', '#푸드스타그램', '#오늘', '#좋아요그램', '#원주맛집', \
             '#원주기업도시맛집', '#먹방스타그램', '#coffeelover', '#원주기업도시카페', '#원주카페', '#원주가볼만한곳',\
              '#원주카페추천', '#원주기업도시핫플', '#원주기업도시마카롱', '#원주마카롱', '#원주메가커피','#전주','#둔산공원',\
              '#봉서중학교','#봉서초등학교','#청완초등학교','#메가커피메뉴','#메가커피원주기업도시점','#원주데이트','#논산',\
              '#논산훈련소','#논산훈련소카페','#연무대','#카페투어','#마라탕','#율하동맛집','#cafeblogger',\
              '#첫줄안녕하세요🙋\u200d♀️','#조아요❤','#팔로우💛','#팔로워🤎','#좋아요👍🏻','#좋아요반사🤍'\
              '#경기도청년노동자통장','#횟집','#봉서중학교'
    

             ] 

instagram_tags = [i for i in hash_tags if i not in stop_words]

# 해쉬태그 빈도수로 상위 400개 검색어 찾기

In [195]:
# 4.해쉬태그 빈도수 끄집어내기 (워드클라우드 작성)

from collections import Counter
count = Counter(instagram_tags) #결과값은 (태그 : 빈도)의 튜플형태로 저장한다 
tag_400 = count.most_common(400) #상위검색어 400개

In [197]:
tag_400

[('#아메리카노', 23),
 ('#스타벅스', 19),
 ('#크로플', 15),
 ('#테이크아웃', 11),
 ('#아이스아메리카노', 11),
 ('#음료추천', 9),
 ('#쿠키프라페', 9),
 ('#바닐라라떼', 9),
 ('#비오는날', 8),
 ('#딸기라떼', 8),
 ('#버블티', 8),
 ('#마카롱', 8),
 ('#라떼', 8),
 ('#메가리카노', 8),
 ('#큐브라떼', 8),
 ('#플레인', 8),
 ('#생크림', 8),
 ('#크림빵', 8),
 ('#초코', 8),
 ('#운동', 8),
 ('#디카페인커피', 7),
 ('#배달카페', 7),
 ('#감자빵', 7),
 ('#아아', 7),
 ('#퐁크러쉬', 6),
 ('#대용량커피', 6),
 ('#국민지원금', 6),
 ('#디저트', 6),
 ('#신메뉴', 6),
 ('#꿀맛', 6),
 ('#점심', 6),
 ('#민트초코', 6),
 ('#토피넛커피쿠키프라페', 6),
 ('#소주한잔', 6),
 ('#사이다', 6),
 ('#얼죽아', 5),
 ('#바나나퐁크러쉬', 5),
 ('#죠리퐁', 5),
 ('#유니콘프라페', 5),
 ('#민트프라페', 5),
 ('#허니브레드', 5),
 ('#출근', 5),
 ('#집밥', 5),
 ('#우유', 5),
 ('#야채', 5),
 ('#토스트', 5),
 ('#야채토스트', 5),
 ('#혼밥', 5),
 ('#카페추천', 4),
 ('#쿠폰', 4),
 ('#가성비', 4),
 ('#로컬푸드', 4),
 ('#조리퐁', 4),
 ('#노마스크', 4),
 ('#선물', 4),
 ('#투썸플레이스', 4),
 ('#존맛', 4),
 ('#토피넛마끼아또', 4),
 ('#힐링', 4),
 ('#당충전', 4),
 ('#아이스라떼', 4),
 ('#음료', 4),
 ('#커피맛집', 4),
 ('#카라멜마끼아또', 4),
 ('#헤이즐넛아메리카노', 4),
 ('#메가커피음료', 3),
 ('#메가커피음료

# 워드클라우드 라이브러리 이용해 시각화

In [198]:
# 5.워드클라우드 라이브러리 불러오기
import matplotlib.pyplot as plt #시각화
from wordcloud import WordCloud #워드클라우드

font_path = "c:/Windows/Fonts/malgun.ttf" 

In [199]:
wc = WordCloud(font_path=font_path,
               background_color="white",
               width=1200, #글자해상도 설정
               height=700 )
cloud = wc.generate_from_frequencies(dict(tag_300)) #단어빈도사전 매소드
                                     
plt.figure(figsize = (30, 20))
plt.imshow(wc,interpolation='bilinear') #이미지 처리방법 : 부드럽게 (bilinear)
plt.axis=('off') #x,y축 없앰
plt.show(cloud)                                     